In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pandas as pd
import os
import imutils
import glob

In [2]:
def show_image(image):
    plt.figure(figsize = (15,5))
    plt.imshow(image,aspect='auto')

In [ ]:
def date_box_finder(image_ip):
    def tup(point):
        return (point[0], point[1]);
    
    # returns true if the two boxes overlap
    def overlap(source, target):
        # unpack points
        tl1, br1 = source;
        tl2, br2 = target;
    
        # checks
        if (tl1[0] >= br2[0] or tl2[0] >= br1[0]):
            return False;
        if (tl1[1] >= br2[1] or tl2[1] >= br1[1]):
            return False;
        return True; 
    
    # returns all overlapping boxes
    def getAllOverlaps(boxes, bounds, index):
        overlaps = [];
        inner = []
        for a in range(len(boxes)):
            if a != index:
                if overlap(bounds, boxes[a]):
                    overlaps.append(a);
        return overlaps;
    
    #img = cv2.imread("input/cheque6.png")
    img = image_ip.copy()
    orig = np.copy(img);
    blue, green, red = cv2.split(img)
    
    def medianCanny(img, thresh1, thresh2):
        median = np.median(img)
        img = cv2.Canny(img, int(thresh1 * median), int(thresh2 * median))
        return img
    
    blue_edges = medianCanny(blue, 0, 1)
    green_edges = medianCanny(green, 0, 1)
    red_edges = medianCanny(red, 0, 1)
    
    edges = blue_edges | green_edges | red_edges
    
    contours,hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL ,cv2.CHAIN_APPROX_SIMPLE)
    
    # go through the contours and save the box edges
    boxes = []; # each element is [[top-left], [bottom-right]];
    hierarchy = hierarchy[0]
    for component in zip(contours, hierarchy):
        currentContour = component[0]
        currentHierarchy = component[1]
        x,y,w,h = cv2.boundingRect(currentContour)
        #if currentHierarchy[3] < 0:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),1)
        boxes.append([[x,y], [x+w, y+h]]);
    
    # filter out excessively large boxes
    filtered = [];
    ####max_area = 10;
    for box in boxes:
        w = box[1][0] - box[0][0];
        h = box[1][1] - box[0][1];
        #if w*h < max_area:
        filtered.append(box);
    boxes = filtered;
    
    # go through the boxes and start merging
    merge_margin = 5;
    
    # this is gonna take a long time
    finished = False;
    highlight = [[0,0], [1,1]];
    points = [[[0,0]]];
    while not finished:
        # set end con
        finished = True;
        # loop through boxes
        index = 0;
        while index < len(boxes):
            # grab current box
            curr = boxes[index];
    
            # add margin
            tl = curr[0][:];
            br = curr[1][:];
            tl[0] -= merge_margin;
            tl[1] -= merge_margin;
            br[0] += merge_margin;
            br[1] += merge_margin;
    
            # get matching boxes
            overlaps = getAllOverlaps(boxes, [tl, br], index);
            # check if empty
            if len(overlaps) > 0:
                # combine boxes
                # convert to a contour
                con = [];
                overlaps.append(index);
                for ind in overlaps:
                    tl, br = boxes[ind];
                    con.append([tl]);
                    con.append([br]);
                con = np.array(con);
    
                # get bounding rect
                x,y,w,h = cv2.boundingRect(con);
                
                if w*h < 8000:
                    # stop growing
                    w -= 1;
                    h -= 1;
                    merged = [[x,y], [x+w, y+h]];
    
                    # highlights
                    highlight = merged[:];
                    points = con;
    
                    # remove boxes from list
                    overlaps.sort(reverse = True);
                    for ind in overlaps:
                        del boxes[ind];
                    boxes.append(merged);
    
                    # set flag
                    finished = False;
                    break;
    
            # increment
            index += 1;
    cv2.destroyAllWindows();
    
        
    lst = []
    for box in boxes:
        lst.append([box[0][0],box[0][1],box[1][0],box[1][1],(box[1][0]-box[0][0]),(box[1][1]-box[0][1])])
    boxes_df = pd.DataFrame(lst,columns=["x1",'y1','x2','y2','width','height'])
    
    #show_image(copy)
    return boxes_df

In [ ]:
def template_matching(template,image):
    template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    template = cv2.Canny(template, 50, 200)
    (tH, tW) = template.shape[:2]
    
    #image = cv2.imread('input/cheque2.png')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    found = None
    resized = imutils.resize(gray, width = int(gray.shape[1] * 1.0))
    r = gray.shape[1] / float(resized.shape[1])
    # if the resized image is smaller than the template, then break
    # from the loop
    edged = cv2.Canny(resized, 50, 200)
    result = cv2.matchTemplate(edged, template, cv2.TM_CCOEFF)
    (_, maxVal, _, maxLoc) = cv2.minMaxLoc(result)
    #if found is None or maxVal > found[0]:
    found = (maxVal, maxLoc, r)
    # unpack the bookkeeping variable and compute the (x, y) coordinates
    # of the bounding box based on the resized ratio
    (_, maxLoc, r) = found
    (startX, startY) = (int(maxLoc[0] * r), int(maxLoc[1] * r))
    (endX, endY) = (int((maxLoc[0] + tW) * r), int((maxLoc[1] + tH) * r))
    cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
    #show_image(image)
    return startX, startY,endX, endY

In [ ]:
template = cv2.imread("templates/date_template.png")
date_roi_dict = {}
for imagePath in glob.glob("input" + "/*.png"):
    #print(os.path.basename(imagePath).split('.')[0])
    image = cv2.imread(imagePath)
    #image_resized = image_resize(image, width=1100)
    #print(imagePath)
    #print(image_resized.shape)
    startX, startY,endX, endY = template_matching(template,image)
    temp_lst = [startX, startY,endX, endY]
    date_roi_dict[imagePath] = temp_lst

In [ ]:
final_date_box_dict = {}
for key,value in date_roi_dict.items():
    #print(key)
    #print(value)
    img = cv2.imread(key)
    #img = image_resize(img, width=1100)
    date_roi_img = img.copy()
    date_roi_final_img = img.copy()
    cv2.rectangle(date_roi_img, (value[0], value[1]), (value[2], value[3]), (0, 255, 0), 2)
    #show_image(date_roi_img)
    #print(img.shape)
    cropped_date = img[value[1]:value[3],value[0]:value[2]]
    boxes_df = date_box_finder(cropped_date)
    max_boxes = boxes_df.max(axis=0)
    min_boxes = boxes_df.min(axis=0)
    x1_ = min_boxes['x1']
    x2_ = max_boxes['x2']
    y1_ = min_boxes['y1']
    y2_ = max_boxes['y2']
    print(x1_,y1_,x2_,y2_)
    #show_image(img)
    #cv2.rectangle(cropped_date, (x1_, y1_), (x2_, y2_), (0, 255, 0), 2)
    #show_image(cropped_date)
    x1 = value[0]+x1_
    y1 = value[1]+y1_
    x2 = value[0]+x2_
    y2 = value[1]+y2_
    #print(x1,y1,x2,y2)
    cv2.rectangle(date_roi_final_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    show_image(date_roi_final_img)
    final_date_box_dict[key] = [x1,y1,x2,y2]

In [ ]:
for k,v in final_date_box_dict.items():
    #print(k)
    #print(v)
    img_itr = cv2.imread(k)
    #print(img_itr.shape)
    #print(v[3]-v[1])
    date_box_height = abs(v[3]-v[1])
    date_box_width = abs(v[2]-v[0])
    upper_cheque_border = int(max(0,v[1]-25))
    right_cheque_border = int(min(img_itr.shape[1],v[2]+50))
    bottom_cheque_border = int(min(img_itr.shape[0],upper_cheque_border+430))
    left_cheque_border = int(max(0,right_cheque_border-1130))
    #print(upper_cheque_border,right_cheque_border,bottom_cheque_border,left_cheque_border,date_box_height,date_box_width)
    date_box_y1_new = v[1] if upper_cheque_border == 0 else 25
    date_box_x2_new = v[2] if right_cheque_border == img_itr.shape[1] else min(1130,right_cheque_border)-50
    date_box_x1_new = date_box_x2_new - date_box_width 
    date_box_y2_new = date_box_y1_new + date_box_height 
    new_date_box_coordinates = [date_box_x1_new,date_box_y1_new,date_box_x2_new,date_box_y2_new]
    #print(new_date_box_coordinates)
    cropped_cheque = img_itr[upper_cheque_border:bottom_cheque_border,left_cheque_border:right_cheque_border]
    #show_image(cropped_cheque)
    #show_image(cropped_cheque[date_box_y1_new:date_box_y2_new,date_box_x1_new:date_box_x2_new])
    logo_lower_bound  = 90
    logo_right_bound = date_box_x1_new
    payee_upper_bound = 80
    payee_lower_bound = 130
    amount_words_first_line_upper_bound = 120
    amount_words_first_line_lower_bound = 170
    amount_words_second_line_upper_bound = 155
    amount_words_second_line_lower_bound = 210
    amount_words_second_line_right_bound = 820
    #amount_words_second_line_lower_bound = 210
    amount_in_number_left_bound = amount_words_second_line_right_bound - 30
    account_number_upper_bound = 190
    account_number_lower_bound = 260
    account_number_right_bound = 450
    singature_box_upper_bound = account_number_upper_bound
    signature_box_lower_bound = 380
    signature_box_left_bound = account_number_right_bound
    micr_band_upper_bound = 360
    #show_image(cropped_cheque[micr_band_upper_bound:,:])
    try:
        if not os.path.exists(os.path.join('segmentation_op',os.path.basename(k).split('.')[0])):
            os.mkdir(os.path.join('segmentation_op',os.path.basename(k).split('.')[0]))
    except Exception as e:
        print(e)
    date_seg_fname = os.path.join('segmentation_op',os.path.basename(k).split('.')[0],'date_seg.png')
    date_seg = cropped_cheque[date_box_y1_new:date_box_y2_new,date_box_x1_new:date_box_x2_new]
    payee_seg_fname = os.path.join('segmentation_op',os.path.basename(k).split('.')[0],'payee_seg.png')
    payee_seg = cropped_cheque[payee_upper_bound:payee_lower_bound,:]
    amount_word_first_seg_fname = os.path.join('segmentation_op',os.path.basename(k).split('.')[0],'amount_word_first_seg.png')
    amount_word_first_seg = cropped_cheque[amount_words_first_line_upper_bound:amount_words_first_line_lower_bound,:]
    
    amount_word_second_seg_fname = os.path.join('segmentation_op',os.path.basename(k).split('.')[0],'amount_word_second_seg.png')
    amount_word_second_seg = cropped_cheque[amount_words_second_line_upper_bound:amount_words_second_line_lower_bound,:amount_words_second_line_right_bound]
    amount_number_seg_fname = os.path.join('segmentation_op',os.path.basename(k).split('.')[0],'amount_number_seg.png')
    amount_number_seg = cropped_cheque[amount_words_second_line_upper_bound:amount_words_second_line_lower_bound,amount_in_number_left_bound:]
    account_number_seg_fname = os.path.join('segmentation_op',os.path.basename(k).split('.')[0],'account_number_seg.png')
    account_number_seg = cropped_cheque[account_number_upper_bound:account_number_lower_bound,:account_number_right_bound]
    singature_seg_fname = os.path.join('segmentation_op',os.path.basename(k).split('.')[0],'singature_seg.png')
    singature_seg = cropped_cheque[singature_box_upper_bound:signature_box_lower_bound,signature_box_left_bound:]
    micr_seg_fname = os.path.join('segmentation_op',os.path.basename(k).split('.')[0],'micr_seg.png')
    micr_seg = cropped_cheque[micr_band_upper_bound:,:]
    logo_seg_fname = os.path.join('segmentation_op',os.path.basename(k).split('.')[0],'logo_seg.png')
    logo_seg = cropped_cheque[:logo_lower_bound,:logo_right_bound]
    
    #show_image(date_seg)
    cv2.imwrite(date_seg_fname,date_seg)
    cv2.imwrite(payee_seg_fname,payee_seg)
    cv2.imwrite(amount_word_first_seg_fname,amount_word_first_seg)
    cv2.imwrite(amount_word_second_seg_fname,amount_word_second_seg)
    cv2.imwrite(amount_number_seg_fname,amount_number_seg)
    cv2.imwrite(account_number_seg_fname,account_number_seg)
    cv2.imwrite(singature_seg_fname,singature_seg)
    cv2.imwrite(micr_seg_fname,micr_seg)
    cv2.imwrite(logo_seg_fname,logo_seg)